In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV

from sklearn import linear_model
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [3]:
data = pd.read_csv('../data/dfLongterm.csv')

In [4]:
data.head()

,fea1,fea2,fea3,fea4,fea5,fea6,fea7,fea8,fea9,fea10,...,fea24,fea25,fea26,fea27,fea28,fea29,fea30,fea31,fea32,fea33
0,137.14,-210.30,70.38,-1.80,13.52,7.67,9.21,-4.23,3.85,-2.07,...,0.33,0.34,0.41,0.43,0.20,0.17,0.26,0.54,0.42,0.45
1,96.00,-321.79,105.26,16.30,33.23,11.20,28.95,12.95,16.30,2.92,...,0.55,0.59,0.41,0.38,0.27,0.22,0.09,0.05,0.12,0.25
2,174.55,-216.06,88.31,-8.95,41.89,18.97,7.79,-2.24,5.94,0.00,...,0.25,0.21,0.44,0.21,0.09,0.18,0.31,0.67,0.40,0.53
3,49.23,-185.45,85.01,-8.47,3.75,-17.62,6.10,-14.41,-8.74,-7.15,...,0.24,0.50,0.53,0.20,0.26,0.09,0.14,0.17,0.31,0.33
4,120.00,-209.14,69.00,30.20,31.84,14.17,22.39,5.16,12.23,0.63,...,0.03,0.06,0.12,0.22,0.23,0.18,0.17,0.17,0.43,0.82


In [5]:
y = pd.read_csv('../data/dfY.csv')

In [6]:
y.head()

,index,song_id,valence_mean,arousal_mean
0,0,2,3.1,3.0
1,1,3,3.5,3.3
2,2,4,5.7,5.5
3,3,5,4.4,5.3
4,4,7,5.8,6.4


In [ ]:
sns.distplot(y['valence_mean'])

In [ ]:
sns.distplot(y['arousal_mean'])

In [ ]:
df = pd.concat([data,y['valence_mean'], y['arousal_mean']], axis=1)

In [ ]:
k = 15 #number of variables for heatmap
corrmat = df.corr()
cols = corrmat.nlargest(k, 'valence_mean')['valence_mean'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [7]:
scaler = StandardScaler()

In [8]:
scaler.fit(data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
data_scaled = scaler.transform(data)

In [10]:
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

In [11]:
data_scaled.head()

,fea1,fea2,fea3,fea4,fea5,fea6,fea7,fea8,fea9,fea10,...,fea24,fea25,fea26,fea27,fea28,fea29,fea30,fea31,fea32,fea33
0,0.655190,-0.076289,-0.775442,-0.034227,-0.765004,0.625078,0.209074,-0.232969,0.167046,0.303801,...,-0.094536,0.119726,0.342517,0.607786,-0.716554,-0.999566,-0.435367,0.917389,0.368432,0.424233
1,-0.697601,-1.237927,0.141235,0.678357,0.547352,0.926323,1.939007,1.843453,1.536770,0.989698,...,1.096746,1.603536,0.342517,0.318272,-0.294819,-0.719813,-1.472541,-1.818019,-1.379786,-0.709320
2,1.885329,-0.136304,-0.304226,-0.315718,1.123963,1.589404,0.084631,0.007548,0.396984,0.588331,...,-0.527729,-0.651855,0.509935,-0.666075,-1.379279,-0.943615,-0.130315,1.643110,0.251884,0.877655
3,-2.235521,0.182628,-0.390953,-0.296821,-1.415522,-1.533136,-0.063474,-1.463352,-1.218080,-0.394467,...,-0.581878,1.069364,1.012190,-0.723977,-0.355067,-1.447171,-1.167490,-1.148123,-0.272581,-0.255899
4,0.091582,-0.064203,-0.811710,1.225590,0.454802,1.179779,1.364115,0.901932,1.088996,0.674927,...,-1.719011,-1.542141,-1.275859,-0.608172,-0.535810,-0.943615,-0.984459,-1.148123,0.426706,2.521308


In [ ]:
def modelTest(clf, train, labels):
    cv = KFold(n_splits=5,shuffle=True,random_state=45).split(train)
    mse = make_scorer(mean_squared_error)
    mse_val_score = cross_val_score(clf, train, labels, cv=cv, scoring=mse)
    scores=[mse_val_score.mean()]
    return scores

In [ ]:
def modelPlot(result_dict):
    result = pd.DataFrame.from_dict(result_dict, orient='index')
    result.columns = ["Mean Squared Error"] 
    result = result.sort(columns=["Mean Squared Error"],ascending=False)
    #print(result)
    result.plot(kind="bar",title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0.5,1])
    return result

In [ ]:
def modelFit(train, labels):
    result_dict = {}
    
    clf = linear_model.LinearRegression()
    result_dict["Linear"] = modelTest(clf, train, labels)
    
    clf = linear_model.Lasso(alpha=1e-4)
    result_dict["Lasso"] = modelTest(clf, train, labels)
    
    clf = linear_model.Ridge()
    result_dict["Ridge"] = modelTest(clf, train, labels)
    
    clf = linear_model.BayesianRidge()
    result_dict["Bayesian Ridge"] = modelTest(clf, train, labels)
    
    clf = linear_model.HuberRegressor()
    result_dict["Huber"] = modelTest(clf, train, labels)
    
    clf = svm.SVR()
    result_dict["SVM RBF"] = modelTest(clf, train, labels)
    
    clf = svm.SVR(kernel="linear")
    result_dict["SVM Linear"] = modelTest(clf, train, labels)
    
    clf = BaggingRegressor()
    result_dict["Bagging"] = modelTest(clf, train, labels)
    
    clf = RandomForestRegressor()
    result_dict["RandomForest"] = modelTest(clf, train, labels)
    
    clf = AdaBoostRegressor()
    result_dict["AdaBoost"] = modelTest(clf, train, labels)
    
    clf = XGBRegressor()
    result_dict["XGBoost"] = modelTest(clf, train, labels)
    
    model_summary = modelPlot(result_dict)
    return model_summary

In [ ]:
modelFit(data_scaled, y['valence_mean'])

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=7)

In [15]:
clf = XGBRegressor()

In [ ]:
clf.fit(X_train, y_train['valence_mean'])

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
mean_squared_error(y_test['valence_mean'], y_pred)

In [ ]:
df_compare = pd.DataFrame({"valence_target": y_test['valence_mean'], "valence_pred": y_pred})

In [ ]:
df_compare.head()

In [ ]:
df_compare['ErrorRate']= df_compare.apply(lambda row: abs(row['valence_pred']-row['valence_target'])/row['valence_target'], axis=1)

In [ ]:
df_compare['ErrorRate'].describe()

# Fine Tune

In [16]:
# Create k-fold
kf = KFold(n_splits=5, shuffle=True, random_state=7).split(X_train)
scorer = make_scorer(mean_squared_error, greater_is_better=True)

In [17]:
def paraSearch(model, x_train, y_train, parameters):
    gs = GridSearchCV(model, parameters, cv=5, scoring=scorer)
    gs.fit(x_train, y_train)
    return gs.best_params_, gs.best_score_, gs.best_estimator_

In [18]:
#XGBRegressor
xgb_params = {
    'colsample_bytree': [0.5, 0.7, 0.9],
    'subsample': [0.5, 0.6, 0.7],
    'learning_rate': [0.075, 0.05, 0.03, 0.01],
    'max_depth': [1, 3, 5, 7, 9, 11, 13],
    'min_child_weight': [1, 3, 5],
    'n_estimators': [50, 100, 150, 200],
    'reg_alpha': [0, 0.1, 0.3, 0.5],
    'reg_lambda': [0.1, 0.5, 0.7, 1],
}

In [19]:
paraSearch(clf, X_train, y_train['valence_mean'], xgb_params)

({'colsample_bytree': 0.7,
  'learning_rate': 0.01,
  'max_depth': 13,
  'min_child_weight': 1,
  'n_estimators': 50,
  'reg_alpha': 0,
  'reg_lambda': 0.5,
  'subsample': 0.5},
 8.4837638608433714,
 XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
        gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=13,
        min_child_weight=1, missing=None, n_estimators=50, nthread=-1,
        objective='reg:linear', reg_alpha=0, reg_lambda=0.5,
        scale_pos_weight=1, seed=0, silent=True, subsample=0.5))

In [ ]:
clf_finetune =  XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
        gamma=0, learning_rate=0.075, max_delta_step=0, max_depth=7,
        min_child_weight=3, missing=None, n_estimators=100, nthread=-1,
        objective='reg:linear', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=0.5)

In [ ]:
clf_finetune.fit(X_train, y_train['valence_mean'])

In [ ]:
y_pred_finetune = clf_finetune.predict(X_test)

In [ ]:
mean_squared_error(y_test['valence_mean'], y_pred_finetune)

In [ ]:
from hpsklearn import HyperoptEstimator

In [ ]:
from hyperopt import hp

In [ ]:
from hpsklearn import any_regressor

In [ ]:
def run_experiments(
        experimental_run,
        dataset,
        model_class=XGBR,
        loss=LOG_LOSS,
        test_metric=accuracy_score,
        random_state=None,
        dataset_name=None):
    """
    Basic experimental framework.
    Parameters
    ----------
    experimental_run : list of tuples
        These tuples should have exactly three members: the first one
        of `grid_search`, `randomized_search`, `hyperopt_search`,
        `skopt_gp_minimize`, `skopt_forest_minimize`, or
        `skopt_forest_gbrt`, the second an appropriate `param_grid`
        dict for that function, and the third a dict specifying
        keyword arguments to the search function.
    dataset : (np.array, iterable)
        A dataset (X, y) where `X` has dimension
        `(n_samples, n_features)` and `y` has
         dimension `n_samples`.
    
    model_class : classifier
        A classifier model in the mode of `sklearn`, with at least
        `fit` and `predict` methods operating on things like
        `X` and `y`.
    loss : function or string
        An appropriate loss function or string recognizable by
        `sklearn.cross_validation.cross_val_score`. In `sklearn`, scores
        are positive and losses are negative because they maximize,
        but here we are minimizing so we always want smaller to mean
        better.
    test_metric : function
        An `sklearn.metrics` function.
    random_state : int
    dataset_name : str or None
        Informal name to give the dataset. Purely for
        book-keeping.
    Returns
    -------
    list of dict
       Each dict is a results dictionary of the sort returned
       by `assess`.
    """                    
    X, y = dataset    
    skf = get_cross_validation_indices(
        X, y, random_state=random_state)        
    all_results = []
    # This loop can easily be parallelized, but doing so can
    # be tricky on some systems, since `cross_val_score`
    # calls `joblib` even if `n_jobs=1`, resulting in
    # nested parallel jobs even if there is no actual
    # parallelization elsewhere in the experimental run.
    for search_func, param_grid, kwargs in experimental_run:
        print(search_func.__name__)
        all_results.append(
            assess(
                X, y,                
                search_func=search_func, 
                model_class=XGBClassifier, 
                param_grid=param_grid,
                xval_indices=skf,
                loss=loss,
                test_metric=test_metric,                
                dataset_name=dataset_name,
                search_func_args=kwargs))
    return all_results
